In [1]:
import utilities as u
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader

# Define Dataset

In [2]:
class TopLevelProofDataset(InMemoryDataset):
    def __init__(self, root='', transform=None, pre_transform=None):
        super(TopLevelProofDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['../top_level_proofs.dataset']
    
    def download(self):
        pass
    
    def process(self):
        data_list = []
        
        for thm, y in data:
            thm = u.process_theorem(thm)
            tree, distinct_features = u.thm_to_tree(thm)
            
            x, edge_index = u.graph_to_data(tree, list(distinct_features))
            
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# SAGEConv Layer

In [3]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

# GNN definition

In [4]:
embed_dim = 128
from torch_geometric.nn import TopKPooling, GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GCNConv(embed_dim, 128)
        self.embedding = torch.nn.Embedding(num_embeddings=len(distinct_features)+1, embedding_dim=embed_dim)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.embedding(x)
        x = x.squeeze(1)      

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

# Data inspections

In [5]:
data = u.make_data()
# data = data[0:2]

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590


In [4]:
distinct_features = set(i for i in range(878))

In [ ]:


# for idx, (thm, _) in enumerate(data):
#     if idx % 1000 == 0:
#         print(f'{idx} / {len(data)}')
#     thm = u.process_theorem(thm)
#     thm_tree, features = u.thm_to_tree(thm)
#     distinct_features = distinct_features.union(features)

In [35]:
test_thm = '(fun (a A B) (a A (a A B)))'
print(test_thm)
thm = u.process_theorem(test_thm)
print(thm)
thm_tree = u.thm_to_tree(thm)
print(thm_tree)

(fun (a A B) (a A (a A B)))
['(', 'fun', '(', 'a', 'A', 'B', ')', '(', 'a', 'A', '(', 'a', 'A', 'B', ')', ')', ')']
(<utilities.data_structures.Tree object at 0x7f41e66b6c50>, {'fun', 'a', 'A', 'B'})


In [36]:
counter = dict()
for _, y in data:
    if y in counter:
        counter[y] += 1
    else:
        counter[y] = 1
counter = list(counter.items())
counter.sort(key=lambda x: x[1], reverse=True)
percentages = [(x, y/len(datapoints)*100) for x,y in counter]
percentages

NameError: name 'data' is not defined

# Create Dataset

In [6]:
dataset = TopLevelProofDataset()
print(dataset)
loader = DataLoader(dataset, batch_size=1, shuffle=True)
for batch in loader:
    print(batch)

TopLevelProofDataset(13677)
Batch(batch=[665], edge_index=[2, 664], x=[665, 1], y=[1])
Batch(batch=[131], edge_index=[2, 130], x=[131, 1], y=[1])
Batch(batch=[231], edge_index=[2, 230], x=[231, 1], y=[1])
Batch(batch=[625], edge_index=[2, 624], x=[625, 1], y=[1])
Batch(batch=[302], edge_index=[2, 301], x=[302, 1], y=[1])
Batch(batch=[472], edge_index=[2, 471], x=[472, 1], y=[1])
Batch(batch=[461], edge_index=[2, 460], x=[461, 1], y=[1])
Batch(batch=[930], edge_index=[2, 929], x=[930, 1], y=[1])
Batch(batch=[177], edge_index=[2, 176], x=[177, 1], y=[1])
Batch(batch=[312], edge_index=[2, 311], x=[312, 1], y=[1])
Batch(batch=[137], edge_index=[2, 136], x=[137, 1], y=[1])
Batch(batch=[115], edge_index=[2, 114], x=[115, 1], y=[1])
Batch(batch=[249], edge_index=[2, 248], x=[249, 1], y=[1])
Batch(batch=[578], edge_index=[2, 577], x=[578, 1], y=[1])
Batch(batch=[117], edge_index=[2, 116], x=[117, 1], y=[1])
Batch(batch=[439], edge_index=[2, 438], x=[439, 1], y=[1])
Batch(batch=[1101], edge_ind

Batch(batch=[345], edge_index=[2, 344], x=[345, 1], y=[1])
Batch(batch=[837], edge_index=[2, 836], x=[837, 1], y=[1])
Batch(batch=[632], edge_index=[2, 631], x=[632, 1], y=[1])
Batch(batch=[228], edge_index=[2, 227], x=[228, 1], y=[1])
Batch(batch=[1077], edge_index=[2, 1076], x=[1077, 1], y=[1])
Batch(batch=[379], edge_index=[2, 378], x=[379, 1], y=[1])
Batch(batch=[168], edge_index=[2, 167], x=[168, 1], y=[1])
Batch(batch=[747], edge_index=[2, 746], x=[747, 1], y=[1])
Batch(batch=[457], edge_index=[2, 456], x=[457, 1], y=[1])
Batch(batch=[413], edge_index=[2, 412], x=[413, 1], y=[1])
Batch(batch=[209], edge_index=[2, 208], x=[209, 1], y=[1])
Batch(batch=[436], edge_index=[2, 435], x=[436, 1], y=[1])
Batch(batch=[124], edge_index=[2, 123], x=[124, 1], y=[1])
Batch(batch=[285], edge_index=[2, 284], x=[285, 1], y=[1])
Batch(batch=[277], edge_index=[2, 276], x=[277, 1], y=[1])
Batch(batch=[121], edge_index=[2, 120], x=[121, 1], y=[1])
Batch(batch=[137], edge_index=[2, 136], x=[137, 1], y

Batch(batch=[245], edge_index=[2, 244], x=[245, 1], y=[1])
Batch(batch=[357], edge_index=[2, 356], x=[357, 1], y=[1])
Batch(batch=[724], edge_index=[2, 723], x=[724, 1], y=[1])
Batch(batch=[203], edge_index=[2, 202], x=[203, 1], y=[1])
Batch(batch=[993], edge_index=[2, 992], x=[993, 1], y=[1])
Batch(batch=[221], edge_index=[2, 220], x=[221, 1], y=[1])
Batch(batch=[272], edge_index=[2, 271], x=[272, 1], y=[1])
Batch(batch=[217], edge_index=[2, 216], x=[217, 1], y=[1])
Batch(batch=[137], edge_index=[2, 136], x=[137, 1], y=[1])
Batch(batch=[476], edge_index=[2, 475], x=[476, 1], y=[1])
Batch(batch=[269], edge_index=[2, 268], x=[269, 1], y=[1])
Batch(batch=[443], edge_index=[2, 442], x=[443, 1], y=[1])
Batch(batch=[1002], edge_index=[2, 1001], x=[1002, 1], y=[1])
Batch(batch=[206], edge_index=[2, 205], x=[206, 1], y=[1])
Batch(batch=[418], edge_index=[2, 417], x=[418, 1], y=[1])
Batch(batch=[159], edge_index=[2, 158], x=[159, 1], y=[1])
Batch(batch=[115], edge_index=[2, 114], x=[115, 1], y

Batch(batch=[339], edge_index=[2, 338], x=[339, 1], y=[1])
Batch(batch=[241], edge_index=[2, 240], x=[241, 1], y=[1])
Batch(batch=[222], edge_index=[2, 221], x=[222, 1], y=[1])
Batch(batch=[355], edge_index=[2, 354], x=[355, 1], y=[1])
Batch(batch=[168], edge_index=[2, 167], x=[168, 1], y=[1])
Batch(batch=[754], edge_index=[2, 753], x=[754, 1], y=[1])
Batch(batch=[80], edge_index=[2, 79], x=[80, 1], y=[1])
Batch(batch=[137], edge_index=[2, 136], x=[137, 1], y=[1])
Batch(batch=[163], edge_index=[2, 162], x=[163, 1], y=[1])
Batch(batch=[280], edge_index=[2, 279], x=[280, 1], y=[1])
Batch(batch=[323], edge_index=[2, 322], x=[323, 1], y=[1])
Batch(batch=[568], edge_index=[2, 567], x=[568, 1], y=[1])
Batch(batch=[441], edge_index=[2, 440], x=[441, 1], y=[1])
Batch(batch=[265], edge_index=[2, 264], x=[265, 1], y=[1])
Batch(batch=[1523], edge_index=[2, 1522], x=[1523, 1], y=[1])
Batch(batch=[423], edge_index=[2, 422], x=[423, 1], y=[1])
Batch(batch=[301], edge_index=[2, 300], x=[301, 1], y=[1

Batch(batch=[246], edge_index=[2, 245], x=[246, 1], y=[1])
Batch(batch=[353], edge_index=[2, 352], x=[353, 1], y=[1])
Batch(batch=[199], edge_index=[2, 198], x=[199, 1], y=[1])
Batch(batch=[349], edge_index=[2, 348], x=[349, 1], y=[1])
Batch(batch=[181], edge_index=[2, 180], x=[181, 1], y=[1])
Batch(batch=[984], edge_index=[2, 983], x=[984, 1], y=[1])
Batch(batch=[477], edge_index=[2, 476], x=[477, 1], y=[1])
Batch(batch=[704], edge_index=[2, 703], x=[704, 1], y=[1])
Batch(batch=[269], edge_index=[2, 268], x=[269, 1], y=[1])
Batch(batch=[356], edge_index=[2, 355], x=[356, 1], y=[1])
Batch(batch=[228], edge_index=[2, 227], x=[228, 1], y=[1])
Batch(batch=[407], edge_index=[2, 406], x=[407, 1], y=[1])
Batch(batch=[217], edge_index=[2, 216], x=[217, 1], y=[1])
Batch(batch=[437], edge_index=[2, 436], x=[437, 1], y=[1])
Batch(batch=[288], edge_index=[2, 287], x=[288, 1], y=[1])
Batch(batch=[93], edge_index=[2, 92], x=[93, 1], y=[1])
Batch(batch=[154], edge_index=[2, 153], x=[154, 1], y=[1])


...

Batch(batch=[212], edge_index=[2, 211], x=[212, 1], y=[1])
Batch(batch=[1165], edge_index=[2, 1164], x=[1165, 1], y=[1])
Batch(batch=[163], edge_index=[2, 162], x=[163, 1], y=[1])
Batch(batch=[385], edge_index=[2, 384], x=[385, 1], y=[1])
Batch(batch=[137], edge_index=[2, 136], x=[137, 1], y=[1])
Batch(batch=[146], edge_index=[2, 145], x=[146, 1], y=[1])
Batch(batch=[415], edge_index=[2, 414], x=[415, 1], y=[1])
Batch(batch=[304], edge_index=[2, 303], x=[304, 1], y=[1])
Batch(batch=[163], edge_index=[2, 162], x=[163, 1], y=[1])
Batch(batch=[311], edge_index=[2, 310], x=[311, 1], y=[1])
Batch(batch=[772], edge_index=[2, 771], x=[772, 1], y=[1])
Batch(batch=[163], edge_index=[2, 162], x=[163, 1], y=[1])
Batch(batch=[675], edge_index=[2, 674], x=[675, 1], y=[1])
Batch(batch=[1216], edge_index=[2, 1215], x=[1216, 1], y=[1])
Batch(batch=[352], edge_index=[2, 351], x=[352, 1], y=[1])
Batch(batch=[169], edge_index=[2, 168], x=[169, 1], y=[1])
Batch(batch=[416], edge_index=[2, 415], x=[416, 1]

Batch(batch=[153], edge_index=[2, 152], x=[153, 1], y=[1])
Batch(batch=[221], edge_index=[2, 220], x=[221, 1], y=[1])
Batch(batch=[1326], edge_index=[2, 1325], x=[1326, 1], y=[1])
Batch(batch=[469], edge_index=[2, 468], x=[469, 1], y=[1])
Batch(batch=[191], edge_index=[2, 190], x=[191, 1], y=[1])
Batch(batch=[706], edge_index=[2, 705], x=[706, 1], y=[1])
Batch(batch=[366], edge_index=[2, 365], x=[366, 1], y=[1])
Batch(batch=[108], edge_index=[2, 107], x=[108, 1], y=[1])
Batch(batch=[199], edge_index=[2, 198], x=[199, 1], y=[1])
Batch(batch=[269], edge_index=[2, 268], x=[269, 1], y=[1])
Batch(batch=[263], edge_index=[2, 262], x=[263, 1], y=[1])
Batch(batch=[225], edge_index=[2, 224], x=[225, 1], y=[1])
Batch(batch=[141], edge_index=[2, 140], x=[141, 1], y=[1])
Batch(batch=[247], edge_index=[2, 246], x=[247, 1], y=[1])
Batch(batch=[122], edge_index=[2, 121], x=[122, 1], y=[1])
Batch(batch=[688], edge_index=[2, 687], x=[688, 1], y=[1])
Batch(batch=[466], edge_index=[2, 465], x=[466, 1], y

Batch(batch=[160], edge_index=[2, 159], x=[160, 1], y=[1])
Batch(batch=[359], edge_index=[2, 358], x=[359, 1], y=[1])
Batch(batch=[337], edge_index=[2, 336], x=[337, 1], y=[1])
Batch(batch=[207], edge_index=[2, 206], x=[207, 1], y=[1])
Batch(batch=[285], edge_index=[2, 284], x=[285, 1], y=[1])
Batch(batch=[180], edge_index=[2, 179], x=[180, 1], y=[1])
Batch(batch=[151], edge_index=[2, 150], x=[151, 1], y=[1])
Batch(batch=[107], edge_index=[2, 106], x=[107, 1], y=[1])
Batch(batch=[289], edge_index=[2, 288], x=[289, 1], y=[1])
Batch(batch=[200], edge_index=[2, 199], x=[200, 1], y=[1])
Batch(batch=[236], edge_index=[2, 235], x=[236, 1], y=[1])
Batch(batch=[461], edge_index=[2, 460], x=[461, 1], y=[1])
Batch(batch=[367], edge_index=[2, 366], x=[367, 1], y=[1])
Batch(batch=[1231], edge_index=[2, 1230], x=[1231, 1], y=[1])
Batch(batch=[213], edge_index=[2, 212], x=[213, 1], y=[1])
Batch(batch=[181], edge_index=[2, 180], x=[181, 1], y=[1])
Batch(batch=[329], edge_index=[2, 328], x=[329, 1], y

Batch(batch=[466], edge_index=[2, 465], x=[466, 1], y=[1])
Batch(batch=[730], edge_index=[2, 729], x=[730, 1], y=[1])
Batch(batch=[583], edge_index=[2, 582], x=[583, 1], y=[1])
Batch(batch=[257], edge_index=[2, 256], x=[257, 1], y=[1])
Batch(batch=[936], edge_index=[2, 935], x=[936, 1], y=[1])
Batch(batch=[477], edge_index=[2, 476], x=[477, 1], y=[1])
Batch(batch=[292], edge_index=[2, 291], x=[292, 1], y=[1])
Batch(batch=[135], edge_index=[2, 134], x=[135, 1], y=[1])
Batch(batch=[1804], edge_index=[2, 1803], x=[1804, 1], y=[1])
Batch(batch=[443], edge_index=[2, 442], x=[443, 1], y=[1])
Batch(batch=[588], edge_index=[2, 587], x=[588, 1], y=[1])
Batch(batch=[363], edge_index=[2, 362], x=[363, 1], y=[1])
Batch(batch=[696], edge_index=[2, 695], x=[696, 1], y=[1])
Batch(batch=[215], edge_index=[2, 214], x=[215, 1], y=[1])
Batch(batch=[522], edge_index=[2, 521], x=[522, 1], y=[1])
Batch(batch=[148], edge_index=[2, 147], x=[148, 1], y=[1])
Batch(batch=[190], edge_index=[2, 189], x=[190, 1], y

Batch(batch=[108], edge_index=[2, 107], x=[108, 1], y=[1])
Batch(batch=[179], edge_index=[2, 178], x=[179, 1], y=[1])
Batch(batch=[204], edge_index=[2, 203], x=[204, 1], y=[1])
Batch(batch=[107], edge_index=[2, 106], x=[107, 1], y=[1])
Batch(batch=[1098], edge_index=[2, 1097], x=[1098, 1], y=[1])
Batch(batch=[163], edge_index=[2, 162], x=[163, 1], y=[1])
Batch(batch=[399], edge_index=[2, 398], x=[399, 1], y=[1])
Batch(batch=[173], edge_index=[2, 172], x=[173, 1], y=[1])
Batch(batch=[258], edge_index=[2, 257], x=[258, 1], y=[1])
Batch(batch=[173], edge_index=[2, 172], x=[173, 1], y=[1])
Batch(batch=[179], edge_index=[2, 178], x=[179, 1], y=[1])
Batch(batch=[246], edge_index=[2, 245], x=[246, 1], y=[1])
Batch(batch=[248], edge_index=[2, 247], x=[248, 1], y=[1])
Batch(batch=[463], edge_index=[2, 462], x=[463, 1], y=[1])
Batch(batch=[100], edge_index=[2, 99], x=[100, 1], y=[1])
Batch(batch=[361], edge_index=[2, 360], x=[361, 1], y=[1])
Batch(batch=[100], edge_index=[2, 99], x=[100, 1], y=[

# Train

In [47]:
batch_size = 1
num_epochs = 10

def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        x = data.x.squeeze(1)

        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        label = label.float()
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    print(loss_all)
    return loss_all / len(dataset)
    
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    a = train()
    #print(a)

1.4492209553718567
0.5943790078163147
-0.7329014092683792
-2.476991791278124
-5.442154627293348
-99.9998552694451
-100.0
-100.0
-100.0
-100.0
